In [33]:
import numpy as np
import openai
import pandas as pd
import os
import sys
import time

# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

In [21]:
# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, 'data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'amazon-beauty/merged_data.csv')
print(f'Data path: {data_path}')

# zero shot save path
ZERO_SHOT_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/all_small_predictions_zero_shot.csv')
print(f'Zero shot save path: {ZERO_SHOT_SAVE_PATH}')

# few shot save path
FEW_SHOT_1_OBS_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/all_small_1_test_predictions_few_shot.csv')
print(f'Few shot save path: {FEW_SHOT_1_OBS_SAVE_PATH}')

Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/merged_data.csv
Zero shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/all_small_predictions_zero_shot.csv
Few shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/all_small_1_test_predictions_few_shot.csv


# Data Overview

In [22]:
# Read the data
data = pd.read_csv(data_path)

# get statistic and first few data of NUM_SAMPLES rows
data.info()
data.head(NUM_EXAMPLES)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rating          34 non-null     float64
 1   verified        34 non-null     bool   
 2   reviewTime      34 non-null     object 
 3   reviewerID      34 non-null     object 
 4   asin            34 non-null     object 
 5   reviewerName    34 non-null     object 
 6   reviewText      34 non-null     object 
 7   summary         34 non-null     object 
 8   unixReviewTime  34 non-null     object 
 9   vote            3 non-null      float64
 10  style           17 non-null     object 
 11  category        34 non-null     object 
 12  tech1           0 non-null      float64
 13  description     34 non-null     object 
 14  fit             0 non-null      float64
 15  title           34 non-null     object 
 16  also_buy        34 non-null     object 
 17  tech2           0 non-null      float

,rating,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
0,5.0,True,2015-09-17,ANV9L0JU6BNL,B000052YAN,Dennis,best floss i've used. does not break as easily...,best floss i've used,2015-09-17,NaN,...,NaN,Reach,[],120.0,"['B01I9TJRN4', 'B003XDVERE', 'B0722XHMGZ', 'B0...",{'\n Product Dimensions: \n ': '1 x 1 x ...,All Beauty,NaN,NaN,5.17
1,5.0,True,2015-09-17,ANV9L0JU6BNL,B000052YAN,Dennis,best floss i've used. does not break as easily...,best floss i've used,2015-09-17,NaN,...,NaN,Reach,[],120.0,"['B01I9TJRN4', 'B003XDVERE', 'B0722XHMGZ', 'B0...",{'\n Product Dimensions: \n ': '1 x 1 x ...,All Beauty,NaN,NaN,5.17
2,2.0,True,2018-03-27,A2TU781PWGS09X,B00006L9LC,Amazon Customer,Doesnt smell,Two Stars,2018-03-27,NaN,...,NaN,Citre Shine,[],1.0,[],"{'ASIN: ': 'B00006L9LC', 'UPC:': '795827187965...",All Beauty,NaN,NaN,23.00
3,2.0,True,2018-03-27,A2TU781PWGS09X,B00006L9LC,Amazon Customer,Doesnt smell,Two Stars,2018-03-27,NaN,...,NaN,Citre Shine,[],1.0,[],"{'ASIN: ': 'B00006L9LC', 'UPC:': '795827187965...",All Beauty,NaN,NaN,23.00
4,5.0,True,2017-02-02,A3A8F2URN7MEPR,B00006L9LC,Sheila T.,My favorite powder!,Five Stars,2017-02-02,NaN,...,NaN,Citre Shine,[],1.0,[],"{'ASIN: ': 'B00006L9LC', 'UPC:': '795827187965...",All Beauty,NaN,NaN,23.00


# Zero-shot (OpenAI API)

In [4]:
%%time

predict_ratings_zero_shot_and_save(data,
                                       columns_for_prediction=ITEM_SIDE,
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=ZERO_SHOT_SAVE_PATH)

System Fingerprint: fp_eeff13170a
Processing item 1/34

Details: Reach Dentotape Waxed Dental Floss with Extra Wide Cleaning Surface for Large Spaces between Teeth, Unflavored, 100 Yards

Predicted Rating: 1.0 stars

----------------

System Fingerprint: fp_eeff13170a
Processing item 2/34

Details: Reach Dentotape Waxed Dental Floss with Extra Wide Cleaning Surface for Large Spaces between Teeth, Unflavored, 100 Yards

Predicted Rating: 1.0 stars

----------------

System Fingerprint: fp_eeff13170a
Processing item 3/34

Details: Citre Shine Moisture Burst Shampoo - 16 fl oz

Predicted Rating: 1.0 stars

----------------

System Fingerprint: fp_eeff13170a
Processing item 4/34

Details: Citre Shine Moisture Burst Shampoo - 16 fl oz

Predicted Rating: 1.0 stars

----------------

System Fingerprint: fp_eeff13170a
Processing item 5/34

Details: Citre Shine Moisture Burst Shampoo - 16 fl oz

Predicted Rating: 1.0 stars

----------------

System Fingerprint: fp_eeff13170a
Processing item 6/3

In [18]:
evaluate_model_predictions_rmse_mae(
    data_path=ZERO_SHOT_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='rating',
    predicted_ratings_column='zero_shot_predicted_rating'
)


RMSE: 3.5732 (95% CI: (3.1436, 3.9779)) ± 0.0041
MAE: 3.2331 (95% CI: (2.7059, 3.7353)) ± 0.0052

First few actual vs predicted ratings:
Actual: 5.0, Predicted: 1.0000
Actual: 5.0, Predicted: 1.0000
Actual: 2.0, Predicted: 1.0000
Actual: 2.0, Predicted: 1.0000
Actual: 5.0, Predicted: 1.0000


# Few-shot (OpenAI API)


+ For each user, we'll use 4 of their ratings as training data to predict ratings for the rest of their products. Finally, we'll evaluate the predictions against the actual ratings to calculate the overall RMSE and MAE.

+ The rating_history_str now includes both the title and the review text for each of the training data rows

# 1 observation per reviewer - Few-shot OpenAI

In [24]:
%%time

predict_ratings_few_shot_and_save(data,
                                      columns_for_training=ITEM_SIDE + USER_SIDE + INTERACTION_SIDE,
                                       columns_for_prediction=ITEM_SIDE,
                                       obs_per_user=1,
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=FEW_SHOT_1_OBS_SAVE_PATH)


System Fingerprint: fp_eeff13170a
Unexpected response for the provided details: Based on the product details provided, the Reach Dentotape Waxed Dental Floss seems to offer several appealing features such as an extra wide cleaning surface, being ADA accepted, and made in the USA. These factors could positively influence the product's rating. Additionally, the product's long length of 100 yards and the brand's reputation for producing dental care products may also contribute to a higher rating. Given the user's previous rating of 5.0 stars for this product, it seems that these features have already had a positive influence on the rating.
Processing user 1/5, item 2/1

User ANV9L0JU6BNL:
Rating History for Prediction: B001QY8QXM (5.0 stars), B0015S8026 (5.0 stars), B01A4XPV56 (2.0 stars), B000052YAN (5.0 stars)
Predicted Item: Reach Dentotape Waxed Dental Floss with Extra Wide Cleaning Surface for Large Spaces between Teeth, Unflavored, 100 Yards (Code: B000052YAN)
Predicted Rating: 0 st

In [37]:
evaluate_model_predictions_rmse_mae(
    data_path=FEW_SHOT_1_OBS_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='few_shot_predicted_rating'
)


RMSE: 4.5403 (95% CI: (3.5214, 5.0000)) ± 0.0084
MAE: 4.3999 (95% CI: (3.2000, 5.0000)) ± 0.0105

First few actual vs predicted ratings:
Actual: 5.0, Predicted: 0.0000
Actual: 2.0, Predicted: 0.0000
Actual: 5.0, Predicted: 0.0000
Actual: 5.0, Predicted: 0.0000
Actual: 5.0, Predicted: 0.0000


# Limitations:

The model might not fully understand the nuanced relationships between products based on titles alone. Additional context or features might be needed for more accurate predictions.
This approach might be computationally expensive and slower than traditional matrix factorization or deep learning-based recommendation models, especially for a small number of users.

# References

+ https://platform.openai.com/docs/api-reference/authentication